# Batch SQL Silver Products

filter(**Bronze_table**) + **Quarentine_table** => **TEMP_Silver**

**TEMP_Silver** => **Silver_table** OR **Quarentine_table**


In [0]:
USE IDENTIFIER(:catalog ||'.'|| :schema)

## Silver Products


In [0]:
-- Upsert Silver_Products:
MERGE INTO silver_products AS T
USING (
  WITH bronze_plus_quarentine AS (
    SELECT 
      Product_ID,
      SKU,
      Name,
      Unit_Price,
      Registration_TimeStamp,
      Operation,
      Moviment_Date
    FROM bronze_products
    -- Filter by day of moviment (moviment_date):
    WHERE Moviment_Date =:moviment_date

    UNION ALL
    -- Union with product in quarentine table (wich is expected to be manual updated):
    SELECT
      Product_ID,
      SKU,
      Name,
      Unit_Price,
      Registration_TimeStamp,
      Operation,
      Moviment_Date
    FROM quarentine_products
  ),


  bronze_plus_quarentine_with_rules AS (
    SELECT *,row_number() OVER(PARTITION BY Product_ID, Registration_TimeStamp ORDER BY Moviment_Date ASC) AS rn,

            -------------------------- Apply rules: 
            CASE WHEN Unit_Price > 0
                THEN 'valid'
                ELSE 'invalid' 
            END AS status
            ---------------------------------------
    FROM bronze_plus_quarentine
  ) 
  SELECT
    Product_ID,
    SKU,
    Name,
    Unit_Price,
    Registration_TimeStamp,
    Operation,
    Moviment_Date
  FROM bronze_plus_quarentine_with_rules
  ------ We just want to keep the valid records to silver table.
  WHERE (Status = 'valid' OR Operation = 'DELETE') AND rn = 1
  --------------------------------------------------------------------------------------------------------------------------
) AS S
  ON T.Product_ID = S.Product_ID AND T.Registration_TimeStamp = S.Registration_TimeStamp
-- INSERT NEW RECORD:
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
21,0,0,21


### Quarentine Silver

In [0]:
MERGE INTO quarentine_products AS T
USING (
  WITH bronze_plus_quarentine AS (
    SELECT
      Product_ID,
      SKU,
      Name,
      Unit_Price,
      Registration_TimeStamp,
      Operation,
      Moviment_Date,
      Source_File
    FROM bronze_products
    -- Filter by day of moviment (moviment_date) and not Product -2 (artificial Product genareted to deal with empty file, meaning: no change in operational database for that day of moviment):
    WHERE Moviment_Date = :moviment_date AND Product_ID <> -2 AND Operation <> 'DELETE'

    UNION ALL
    -- Union with Product in quarentine table (wich is expected to be manual updated):
    SELECT
      Product_ID,
      SKU,
      Name,
      Unit_Price,
      Registration_TimeStamp,
      Operation,
      Moviment_Date,
      Source_File
    FROM quarentine_products
  ),

  bronze_plus_quarentine_with_rules AS (
    SELECT *,

            -------------------------- Apply rules: 
            CASE WHEN Unit_Price > 0
                THEN 'valid'
                ELSE 'invalid' 
            END AS status
            ---------------------------------------
    FROM bronze_plus_quarentine
  ) 
  SELECT *
  FROM bronze_plus_quarentine_with_rules
  ------ We just want to keep the invalid records to quarentine table.
  WHERE Status = 'invalid' 
  --------------------------------------------------------------------------------------------------------------------------
) AS S

  ON T.Product_ID = S.Product_ID AND T.Registration_TimeStamp = S.Registration_TimeStamp 
-- UPDATE TO LAST RECORD:
WHEN MATCHED THEN UPDATE SET *
-- INSERT NEW RECORD:
WHEN NOT MATCHED THEN INSERT *
-- Deleted retified registers:
WHEN NOT MATCHED BY SOURCE THEN DELETE

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


In [0]:
UPDATE IDENTIFIER(:catalog ||'.control.control_tables_moviments')
SET MOVIMENT_DATE = :moviment_date
WHERE Project_Name = :project
  AND Table_name = 'silver_products'